In [1]:
ONNX_PATH = "LinearBlend_v001.ort"
HEX_CONTENT_PATH = "LinearBlend_v001.txt"

with open(ONNX_PATH, "rb") as fread:
    model_in_bytes = fread.read()
    print("type(model_in_bytes):", type(model_in_bytes))
    model_in_bytes_hex = model_in_bytes.hex()
    print("type(model_in_bytes_hex):", type(model_in_bytes_hex))
    print(len(model_in_bytes_hex))

with open(ONNX_PATH, "rb") as fread, open(HEX_CONTENT_PATH, "w") as fwrite:
    model_in_bytes = fread.read()
    model_in_bytes_hex = model_in_bytes.hex()
    fwrite.write(model_in_bytes_hex)

type(model_in_bytes): <class 'bytes'>
type(model_in_bytes_hex): <class 'str'>
6880
